# Sentimetal Analysis Training Module
* CSCI-4930/5930 ML Fall 2021
* Author: Anurag Kurumeti

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import json
import gzip
num_gpus_available = len(tf.config.experimental.list_physical_devices('GPU'))

In [3]:
!pip install transformers

In [4]:
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

In [5]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

In [6]:
def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

In [7]:
review_dataset = getDF('/content/gdrive/MyDrive/ML_Course_Project/Magazine_Subscriptions.json.gz')

In [8]:
review_dataset.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,5.0,9,False,"11 8, 2001",AH2IFH762VY5U,B00005N7P0,ted sedlmayr,"for computer enthusiast, MaxPC is a welcome si...","AVID READER SINCE ""boot"" WAS THE NAME",1005177600,NaN,NaN
1,5.0,9,False,"10 31, 2001",AOSFI0JEYU4XM,B00005N7P0,Amazon Customer,Thank god this is not a Ziff Davis publication...,The straight scoop,1004486400,NaN,NaN
2,3.0,14,False,"03 24, 2007",A3JPFWKS83R49V,B00005N7OJ,Bryan Carey,Antiques Magazine is a publication made for an...,"Antiques Magazine is Good, but not for Everyone",1174694400,{'Format:': ' Print Magazine'},NaN
3,5.0,13,False,"11 10, 2006",A19FKU6JZQ2ECJ,B00005N7OJ,Patricia L. Porada,This beautiful magazine is in itself a work of...,THE DISCERNING READER,1163116800,{'Format:': ' Print Magazine'},NaN
4,5.0,NaN,True,"07 14, 2014",A25MDGOMZ2GALN,B00005N7P0,Alvey,A great read every issue.,Five Stars,1405296000,NaN,NaN


In [9]:
product_dataset = getDF('/content/gdrive/MyDrive/ML_Course_Project/meta_Magazine_Subscriptions.json.gz')

In [10]:
product_dataset.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,"[Magazine Subscriptions, Professional & Educat...",,[REASON is edited for people interested in eco...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYLE, B01MCU84LB, B000UHI2LW, B01AKS14A...",,Reason Magazine,[],[],"[B002PXVYLE, B000UHI2LW, B01MCU84LB, B002PXW18...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7NQ,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
1,"[Magazine Subscriptions, Arts, Music &amp; Pho...",,[Written by and for musicians. Covers a variet...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYGE, B0054LRNC8, B000BVEELE, B00006KC3...",,String Letter Publishers,[],742 in Magazine Subscriptions (,"[B002PXVYGE, B0054LRNC8, B00006L16A, 171906487...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7OC,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
2,"[Magazine Subscriptions, Fashion &amp; Style, ...",,[Allure is the beauty expert. Every issue is f...,,"<span class=""a-size-medium a-color-secondary""","[B001THPA4O, B002PXVZWW, B001THPA1M, B001THPA1...",,Conde Nast Publications,[],[],"[B002PXVZWW, B001THPA4O, B001THPA1M, B01N819UD...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7OD,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
3,"[Magazine Subscriptions, Sports, Recreation & ...",,[FLIGHT JOURNAL includes articles on aviation ...,,"<span class=""a-size-medium a-color-secondary""","[B07JVF7QW4, B00ATQ6FPY, B002G551F6, B00008CGW...",,AirAge Publishing,[],[],"[B002G551F6, B00ATQ6FPY, B00005N7PT, B001THPA2...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7O9,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,"[Magazine Subscriptions, Professional & Educat...",,[RIDER is published for the road and street ri...,,"<span class=""a-size-medium a-color-secondary""","[B002PXVYD2, B01BM7TOU6, B000060MKJ, B000BNNIG...",,EPG Media & Specialty Information,[],[],"[B01BM7TOU6, B000060MKJ, B002PXVYD2, B000BNNIG...","{'Format:': 'Print Magazine', 'Shipping: ': 'C...",Magazine Subscriptions,,,,B00005N7O6,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [11]:
verified_review_dataset = review_dataset[review_dataset['verified']]
verified_review_dataset = verified_review_dataset[['overall','reviewText']]
verified_review_dataset

,overall,reviewText
4,5.0,A great read every issue.
5,3.0,This magazine was great for the times but as w...
6,5.0,I've read Maximum PC (MPC) for many years. The...
7,5.0,We ordered this magazine for our grandson (the...
8,4.0,I have subscribed to the nook version for a fe...
...,...,...
89684,5.0,This was a nice surprise for my boyfriend. He ...
89685,1.0,Magazine looks like it is printed on recycled ...
89686,5.0,cant go wrong with an SI subscription\nvery pl...
89687,5.0,This magazine is by far my all time favorite o...


In [12]:
verified_review_dataset = verified_review_dataset.dropna()
verified_review_dataset = verified_review_dataset

In [13]:
X = verified_review_dataset.iloc[:,1]
y = verified_review_dataset['overall'] - 1

In [14]:
from sklearn.model_selection import train_test_split
X_train,X_val,y_train,y_val = train_test_split(X.values.tolist(),y.tolist(),test_size = 0.2)
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size = 0.25)

In [15]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [16]:
train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=128)
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings),y_train))
val_encodings = tokenizer(X_val, truncation=True, padding=True, max_length=128)
val_dataset = tf.data.Dataset.from_tensor_slices((dict(val_encodings),y_val))

In [17]:
print(train_dataset)
print(val_dataset)

<TensorSliceDataset shapes: ({input_ids: (128,), attention_mask: (128,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float32)>
<TensorSliceDataset shapes: ({input_ids: (128,), attention_mask: (128,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float32)>
<TensorSliceDataset shapes: ({input_ids: (128,), attention_mask: (128,)}, ()), types: ({input_ids: tf.int32, attention_mask: tf.int32}, tf.float32)>


In [18]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=5)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'activation_13', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use i

In [19]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  3845      
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
Total params: 66,957,317
Trainable params: 66,957,317
Non-trainable params: 0
_________________________________________________________________


In [20]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
model.compile(optimizer=optimizer, loss=model.compute_loss, metrics=['accuracy'])

In [21]:
optimizer.get_config()

{'amsgrad': False,
 'beta_1': 0.9,
 'beta_2': 0.999,
 'decay': 0.0,
 'epsilon': 1e-08,
 'learning_rate': 5e-05,
 'name': 'Adam'}

In [22]:
model.fit(train_dataset.shuffle(100).batch(32),
          epochs=2,
          batch_size=32,
          validation_data=val_dataset.shuffle(100).batch(32))

Epoch 1/2
1100/1100 [==============================] - 13265s 12s/step - loss: 0.7338 - accuracy: 0.7289 - val_loss: 0.7000 - val_accuracy: 0.7245
Epoch 2/2
1100/1100 [==============================] - 13278s 12s/step - loss: 0.6037 - accuracy: 0.7714 - val_loss: 0.7136 - val_accuracy: 0.7303


In [23]:
model.save_pretrained("/content/gdrive/MyDrive/ML_Course_Project/sentiment")

In [26]:
test = []
for test_sentence in X_test:
  test_input = tokenizer.encode(test_sentence, truncation=True, padding=True, return_tensors="tf")
  test.append(test_input)
test

[<tf.Tensor: shape=(1, 81), dtype=int32, numpy=
 array([[  101,  2026,  3129,  2003,  2126,  2046,  2974,  1010,  2061,
          1045,  2288,  2023,  2005,  2032,  1012,  2009,  2001,  1037,
          2307,  3976,  1010,  2061,  1045,  6618,  2057,  2071,  4638,
          2009,  2041,  1012,  2009,  1005,  1055,  3492, 15708,  1012,
          2002,  2288,  2010,  2034,  3277,  2023,  2733,  1998,  2038,
          2525,  3191,  1037,  2843,  1997,  2009,  1012,  3492,  5875,
          1012,  2009,  1005,  1055,  2288,  2673,  2013,  2079,  1011,
          2009,  1011,  4426,  2188,  4784,  2000,  4790,  2006,  1996,
          6745,  2966,  6786,  1998,  3005, 10449,  2068,  1012,   102]],
       dtype=int32)>, <tf.Tensor: shape=(1, 26), dtype=int32, numpy=
 array([[  101,  2025,  2066,  1996,  3147,  2694,  5009,  1045,  3473,
          2039,  2007,  2134,  1005,  1056,  5949,  2115,  2769,  1012,
          2053,  2154, 26213,  1012, 13044,  1012,  1012,   102]],
       dtype=int32)>, 

In [67]:
test_output = []
for test_input in test:
  tf_output = model.predict(test_input)[0]
  tf_prediction = tf.nn.softmax(tf_output, axis=1)
  label = tf.argmax(tf_prediction, axis=1)
  test_output.append(label.numpy()[0])

In [72]:
test_output = [float(x) for x in test_output]

In [74]:
accuracy_function = tf.keras.metrics.Accuracy()
accuracy_function.update_state(y_test, test_output)
accuracy = accuracy_function.result().numpy()
accuracy

0.7371024

In [78]:
loss_function = tf.keras.losses.MeanSquaredError()
loss = loss_function(y_test, test_output).numpy()
loss

0.6010915